In [21]:
import sys
sys.path.append("/storage/BonnieH/selection_project/helper_functions")
from ABC_functions import *

In [22]:
# Code to write example loci to file:

def WriteExampleLoci(inFile, outFile):
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/merged_filtered.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/example_loci.txt'

    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    
    codis_list = [('chr2', 1489651), ('chr5', 123775553), ('chr5', 150076322), ('chr7', 84160225), ('chr8', 124894863), \
                  ('chr11', 2171086), ('chr13', 82148025), ('chr16', 86352702), ('chr18', 63281667)]
        
    disease_list = [('chr19', 45770205), ('chr19', 13207859), ('chr13', 70139384), ('chr5', 146878729)]
    
    early_onset_list = [('chr6', 45315923), ('chr6', 45362057), ('chr6', 45474374), ('chr6', 45478639), \
                       ('chr6', 45506657), ('chr6', 45515702), ('chr6', 45523256), ('chr2', 176089200), \
                       ('chr2', 176094386)]
                  
    codisFile = '/storage/BonnieH/selection_project/annotations/hg38_codis.bed'
    diseaseFile = '/storage/BonnieH/selection_project/annotations/hg38_disease.txt'
    earlyonsetFile = '/storage/BonnieH/selection_project/annotations/hg38_earlyonset.txt'

    codis = open(codisFile, 'r')
    codis_dic = {}
    for line in codis:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        codis_dic[(chrom, start)] = name
    codis.close()
        
    disease = open(diseaseFile, 'r')
    disease_dic = {}
    for line in disease:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        disease_dic[(chrom, start)] = name
    disease.close()

    early = open(earlyonsetFile, 'r')
    early_dic = {}
    for line in early:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[4]
        early_dic[(chrom, start)] = name
    early.close()

    Lines = allele_freqs_file.readlines()
    
    for line in Lines:
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        per = int(info[3])
        freqs = info[4]
            
        # Get info about reference allele (length in base pairs, repeat units)
        ref_length_bp = end - start + 1
        ref_length_ru = int(ref_length_bp/per)
            
        tup = (chrom, start)
            
            
        if (tup in disease_dic) or (tup in early_dic) or (tup in codis_dic):
            print(tup)
            results.write(line)
            '''
            elif per == 2 and ref_length_ru >= 11:
                results.write(line)
                
            elif per == 3 and ref_length_ru >= 5:
                results.write(line)
                
            elif per == 4 and ref_length_ru >= 7:
                results.write(line)
            '''
    results.close()        
    allele_freqs_file.close()
    print("Done")

In [23]:
# Code to filter out short loci based on optimal allele length threshold 

def FilterShortLoci(inFile, outFile):
    
    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    
    num_lines_before_filt = 0
    num_lines_after_filt = 0
    
    for line in allele_freqs_file:
        num_lines_before_filt = num_lines_before_filt + 1
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        freqs = info[3]
        per = int(info[4])
        if freqs == '.':
            print(line)
        
        #  Only continue if freqs != '.'
        if freqs != '.' : 
            
            # Get info about reference allele (length in base pairs, repeat units)
            ref_length_bp = end - start + 1
            ref_length_ru = int(ref_length_bp/per)
            
            optimal_ru = Process_Freqs(freqs, per, end, start, False)
            
            if per == 3 and optimal_ru == 3:
                print(line)
            if per == 1 and optimal_ru >= 20:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            elif per == 2 and optimal_ru >= 11:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1 
                
            elif per == 3 and optimal_ru >= 5:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            elif per == 4 and optimal_ru >= 7:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            # Adding RUNX2 tracts?
            #elif chrom == "chr6" and start == 45442365:
             #   results.write(line)
                
            #elif chrom == "chr6" and start == 45457637:
             #   results.write(line)
            
    results.close()        
    allele_freqs_file.close()
    print("Done")
    print(num_lines_before_filt)
    print(num_lines_after_filt)

In [24]:
def main():
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_per_1234.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_filt_per_1234.txt'
    FilterShortLoci(inFile, outFile)

In [25]:
%%time
if __name__ == '__main__':
    main()

1	186336	186344	0:7168	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	MIR6859-2

1	877282	877290	-3:1,0:7525,3:2	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AL669831.6

1	904440	904448	0:7510	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AL645608.6

1	1004540	1004548	0:7526	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	ISG15

1	1600168	1600176	0:7386	3	CCG	0.0	0.0	0.0	0.0	0.0	1.0	FNDC10

1	1955701	1955709	0:7526	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	CFAP74

1	2221537	2221545	0:7528	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	SKI

1	2502974	2502982	0:7526	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	PLCH2

1	3727198	3727206	0:7528	3	ATC	1.0	0.0	0.0	0.0	0.0	0.0	TP73

1	4073400	4073408	0:7516	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AL805961.2

1	4144358	4144366	0:3480,21:1,24:1	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	EEF1DP6

1	4639072	4639080	0:7528	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AJAP1

1	4767382	4767390	-3:18,0:7430,21:1,24:1	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	Z98886.1

1	4779236	4779244	-3:2,0:7528	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	Z98886.1

1	4796715	4796723	-3:1,0:7473	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	Z98886.1

1	

1	35269452	35269460	0:7528	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	ZMYM4

1	35313209	35313217	0:7524	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	RF00091

1	35375130	35375138	0:7496	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	ZMYM4-AS1

1	35436663	35436671	-3:1,0:7527	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	KIAA0319L

1	35529069	35529077	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AC004865.1

1	35530943	35530951	0:7530	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	AC004865.1

1	35845781	35845789	-3:30,0:7500	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AGO4

1	36163679	36163687	0:7528	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	MAP7D1

1	36413092	36413100	0:7528	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	SNORA63C

1	36459753	36459761	0:7528	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	MRPS15

1	36611436	36611444	0:7528	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	FTLP18

1	36905148	36905156	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	GRIK3

1	36972937	36972945	0:7524	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	GRIK3

1	36985933	36985941	0:7528	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	GRIK3

1	37125255	37125263	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL031430.1


1	76227025	76227033	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	ST6GALNAC3

1	76437895	76437903	0:7522	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AC103592.1

1	76441017	76441025	0:7530	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	AC103592.1

1	76773865	76773873	0:7530	3	ACT	0.0	0.0	0.0	0.0	0.0	1.0	LINC02567

1	76808869	76808877	0:7522	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	LINC02567

1	76823777	76823785	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	ST6GALNAC5

1	76860426	76860434	0:7526	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	ST6GALNAC5

1	77038885	77038893	0:7528	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	MIR7156

1	77360001	77360009	0:7530	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	AC095030.1

1	77366098	77366106	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AK5

1	77867927	77867938	-3:5867,0:1663	3	ATC	0.0	1.0	0.0	0.0	0.0	0.0	HSPE1P25

1	78044197	78044205	0:7530	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	AC103591.2

1	78286165	78286173	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	PTGFR

1	78338518	78338526	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AC096531.1

1	78355378	78355386	0:7526	3	AAG	0.0	0.0	0.0	

1	145512721	145512729	0:7524	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AC239860.1

1	146035984	146035992	-3:1,0:7529	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	HJV

1	146190726	146190734	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	NBPF10

1	146306435	146306443	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	NUDT4P2

1	146745462	146745470	0:7530	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	HYDIN2

1	147461585	147461596	-3:3742,-6:66,0:3716	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	OR13Z2P

1	147547699	147547707	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	BCL9

1	147788987	147788995	0:7526	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AC241644.2

1	148085908	148085916	-3:1,0:7317	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	PDE4DIPP1

1	148352980	148352988	0:7516	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	AC245100.7

1	148641442	148641450	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	NBPF14

1	148757218	148757226	0:7528	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	NUDT4B

1	148926513	148926521	0:7526	3	ATC	0.0	1.0	0.0	0.0	0.0	0.0	PDE4DIP

1	149203760	149203768	0:7530	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	AC245297.3

1	149429579	149429587

1	188844159	188844167	0:7525,3:1	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL691515.2

1	189027337	189027345	0:7524	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	CLPTM1LP1

1	189107158	189107166	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	GAPDHP75

1	189607548	189607556	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	RNA5SP73

1	189751206	189751214	0:7526	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	LINC01701

1	189828598	189828606	-3:1,0:7527	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL591504.2

1	190113274	190113282	0:7530	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	BRINP3

1	190243595	190243603	0:7530	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	BRINP3

1	190322831	190322839	0:7528	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	AL354771.1

1	191430432	191430440	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01680

1	191794929	191794937	0:7526	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL359081.2

1	191972350	191972358	0:7530	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL359081.1

1	192110223	192110231	0:7528	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	RGS18

1	192180215	192180223	0:7526	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	RGS18

1	192192409	192192417	

...

19	36403157	36403165	0:7471,3:1	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	ZFP82

19	37154543	37154551	-3:4,0:7510	3	AAC	0.0	1.0	0.0	0.0	0.0	0.0	AC012309.2

19	37211688	37211696	0:7514	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	ZNF585B

19	37822201	37822209	0:7512	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AC016582.3

19	37892399	37892407	-3:1,0:7512,12:1	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	WDR87

19	38305397	38305405	0:7504	3	CCG	0.0	0.0	0.0	1.0	0.0	0.0	C19orf33

19	38315890	38315898	0:7512	3	CCG	0.0	0.0	1.0	0.0	0.0	0.0	YIF1B

19	38354974	38354982	0:7512	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	CATSPERG

19	38527524	38527532	-3:1,0:7507	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	RYR1

19	38565382	38565390	0:7486	3	CCG	1.0	0.0	0.0	0.0	0.0	1.0	RYR1

19	38644355	38644363	0:7508	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	ACTN4

19	38827668	38827676	0:7510	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	ECH1

19	39000633	39000641	0:7510	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	FBXO17

19	39219806	39219814	0:7440	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	SYCN

19	39795728	39795760	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	


20	5627660	5627668	0:7506	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	GPCPD1

20	5883434	5883442	0:7506	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	RNU1-55P

20	5945827	5945835	0:7504	3	AAT	0.0	1.0	0.0	0.0	1.0	0.0	TRMT6

20	6080588	6080596	0:7506	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	AL118505.1

20	6897671	6897679	0:7506	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	BMP2

20	7135123	7135131	0:7504	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	AL096799.1

20	7172688	7172696	0:7506	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01428

20	7173725	7173733	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	LINC01428

20	7322278	7322286	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	LINC01751

20	7714307	7714315	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	RN7SL547P

20	7806375	7806383	0:7504	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL031679.1

20	7866765	7866773	0:7506	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	AL021879.1

20	7907086	7907094	0:7502	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	HAO1

20	8242801	8242809	0:7502	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	PLCB1-IT1

20	8373389	8373397	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	PLCB1

20	8432027	8

20	50848049	50848057	0:7477,3:27	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	TMSB4XP6

20	51009570	51009578	0:7506	3	AGG	1.0	0.0	0.0	0.0	0.0	1.0	KCNG1

20	51052377	51052385	0:7502	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	KCNG1

20	51239207	51239215	0:7506	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	RPSAP1

20	51670950	51670958	0:7504	3	ATC	0.0	0.0	0.0	0.0	0.0	1.0	ATP9A

20	51740566	51740574	.	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL138807.1

20	51750169	51750177	0:7500	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL138807.1

20	51804412	51804420	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	SALL4

20	51982541	51982549	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	RNU7-6P

20	52131075	52131083	0:7496	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	ERP29P1

20	52245740	52245748	0:7506	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01524

20	52371078	52371086	0:7506	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01524

20	52463314	52463322	0:7506	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL109610.1

20	52709130	52709138	-3:1,0:7505	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	AL031674.1

20	52725696	52725704	0:7504	3	ACC	0.0	0.0	0.0	0.0

21	44861716	44861724	0:7504	3	AGG	0.0	1.0	0.0	0.0	0.0	0.0	PTTG1IP

21	45405399	45405407	0:7472	3	CCG	1.0	0.0	0.0	0.0	0.0	1.0	COL18A1

21	45442683	45442691	-3:2,0:7460	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	COL18A1

21	45442794	45442802	-3:85,0:7371	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	COL18A1

22	10755986	10755994	0:28	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	RF00004

22	11051540	11051548	0:48	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	CU104787.1

22	12604706	12604714	-3:58,0:1286	3	AAG	0.0	0.0	0.0	0.0	0.0	1.0	FRG1GP

22	15241090	15241133	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	RF00026

22	15573612	15573620	0:7472	3	CCG	0.0	0.0	0.0	0.0	0.0	1.0	AP000533.1

22	16433735	16433743	-3:1,0:7253	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	PABPC1P9

22	16719330	16719338	0:7500	3	ACT	0.0	0.0	0.0	0.0	0.0	1.0	AC005301.2

22	16805556	16805564	0:7500	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	XKR3

22	17213584	17213592	0:7502	3	AAG	0.0	1.0	0.0	0.0	0.0	0.0	FAM32BP

22	17312463	17312471	0:7502	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	CECR9

22	17415220	17415228	-3:2,0:7500	3	AAG	0.0	0.0	

22	47446777	47446785	-3:1,0:7501	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	LINC01644

22	47469558	47469566	0:7500	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	LINC01644

22	47506879	47506887	0:7500	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	LINC01644

22	48314344	48314352	0:7502	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	MIR3201

22	48538274	48538282	0:7500	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	Z84468.1

22	48575683	48575691	0:7192,36:1,42:1	3	CCG	0.0	0.0	0.0	0.0	0.0	1.0	TAFA5

22	49003290	49003298	0:6472	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL078622.1

22	49156437	49156445	0:7498	3	AGG	0.0	0.0	0.0	0.0	0.0	1.0	Z82202.2

22	49273681	49273689	0:7500	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	Z82202.1

22	49563427	49563435	0:7502	3	AAT	0.0	0.0	0.0	0.0	0.0	1.0	Z97192.2

22	49595453	49595461	0:7502	3	ACC	0.0	0.0	0.0	0.0	0.0	1.0	Z97192.2

22	49716424	49716432	0:7500	3	AGC	0.0	0.0	0.0	0.0	0.0	1.0	AL023802.1

22	49731430	49731438	0:7502	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL023802.1

22	49747659	49747667	-3:1,0:7481	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	RN7SKP252

22	49774647	49774655	0